# Notebook to Move final dcm files selected for processing to Project Folder

In [2]:
import shutil
import pickle
import pandas as pd
from l3finder.ingest import *
import os
import multiprocessing
from multiprocessing import get_context
from multiprocessing import set_start_method
from tqdm.notebook import tqdm
from investigate import *

# Custom functions
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):        
    with open(filename, 'rb') as input:
        return pickle.load(input)

In [3]:
cwd = os.getcwd()
output = '/tf/pickles/v5_8pts'
data = '/tf/data'
target = '/tf/output'
dump=1

### Open final df and reduce it to only the dump you are processing

In [5]:
df_final = load_object(os.path.join(output,'df_final_dump_1_8pats.pkl'))
subjects = load_object(os.path.join(output,'subjects_final_dump_1_8pats.pkl'))

In [7]:
#df_final = df_final[df_final['dump']==dump]
#subjects = [s for s in subjects if s.id_ in df_final['ID'].values]

print('Len of df_final for this dump:  ', len(df_final))
print('Len of subjects for this dump:  ', len(subjects))

Len of df_final for this dump:   8
Len of subjects for this dump:   8


# File Mover

In [8]:
file_mover = functools.partial(
        move_subject,
        df_final=df_final,
        target = target)

In [9]:
df_final = df_final[df_final['ID'].isin([s.id_ for s in subjects])]
print('Len of df: ', len(df_final))
print('Len of sub: ', len(subjects))

Len of df:  8
Len of sub:  8


In [10]:
if __name__=='__main__':
    # Find series images
    print("Moving valid subjects")
    
    def pool_mover(pool, func, candidates):
        return [a for a in tqdm(pool.imap_unordered(func, candidates),total=len(candidates))]
        
    print('Filtering series using ', 10, ' cores:')
        
    with get_context("spawn").Pool(processes=10) as p:
        result_list = pool_mover(p, file_mover,subjects)
        p.close()
        p.join()
        
    df_final['Processed'] = result_list

Moving valid subjects
Filtering series using  10  cores:


In [11]:
display(df_final.head(10))

,ID,Axial,Sagittal,Overlap,MissingScore,PairValidity,AxSlices,SagSlices,AxThick,SagThick,Processed
0,Z923887,Z923887-SE-4-Axial_Body_5.0,None,None,1.0,None,84,None,5.0,None,1
1,Z1107859,Z1107859-SE-4-Axial_Std_3.0,None,None,1.0,None,86,None,3.0,None,1
2,Z1312350,Z1312350-SE-5-Axial_Body_5.0,None,None,1.0,None,51,None,5.0,None,1
3,Z934614,Z934614-SE-4-Standard_Axial,None,None,1.0,None,74,None,5.0,None,1
4,Z1146560,Z1146560-SE-2-Stnd_Pediatric_Axial_Stnd,None,None,1.0,None,72,None,5.0,None,1
5,Z719159,Z719159-SE-4-Std_Axial_Body_Std_5.0,None,None,1.0,None,70,None,5.0,None,1
6,Z1141194,Z1141194-SE-2-Stnd_Pediatric_5.0_CE,Z1141194-SE-5-Stnd_Pediatric_3.0_CE,1,1.0,2,86,107,5.0,3,1
7,Z1199636,Z1199636-SE-2-Body_Std._Axial_AIDR_-_H_5.0,None,None,1.0,None,76,None,5.0,None,1


In [12]:
save_object(df_final,os.path.join(output,'df_final_v5_8pts_processed.pkl'))